# 常见的共识算法

达成共识需要满足以下两个条件：  
- 选出的记账权威必须是诚实节点
- 该节点创建的区块必须被其他诚实节点验证通过

### 工作量证明(POW)原理

求满足$H(z)<y$的$z$，假设数据$z$的格式是固定的，由一个固定的字符串s加上一个随机数nonce组成的，随机数nonce从0开始取值，每次增加1，直到找到满足$H(z)<y$的nonce。

In [1]:
from simchain import sha256d
# 给定一个字符串s
s = "I love block chain"
# 初始化nonce
nonce = 0
for _ in range(10):
    # 组合字符串s与随机数nonce为z
    z = s + str(nonce)
    # 计算z的哈希值
    print("{0}->{1}".format(z, sha256d(z)))
    # nonce增加1
    nonce += 1

2019-06-13 23:03:59,628 - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


I love block chain0->3dab2633d7cc5de223a2a616725fc52cb5adb92885cc0abc9bb181a5ccc79a98
I love block chain1->9c7ec63641d863b674562529f0094d644628fa9c2fe717c1ea7284dd42239bdf
I love block chain2->1379414848279f4827f57927026b02007f1bec2f21433a4b29ccec4fa83bef69
I love block chain3->5b335251ef9dfc4861240bfa34daba259c554eb2a106a76b45c04dcf2e007340
I love block chain4->91cf44991b08fb49c12c73f5f30345ddd6690942fe2bf2e70d247efe3a55de11
I love block chain5->5f389676683ea636e52f96ffa9f8c7d578444648f03fab01bc2fd6e40cef9fb4
I love block chain6->830eb5892a5fd6e6978fd84d773b2b0c88dd584c14556428e95b91037d872d3d
I love block chain7->c4f39c4c3befaa93d6cae5cbb2d95ffac22ed82421fe171ede9bd6e2827cbc9a
I love block chain8->596794760f603a41a7f1231712db6795c3a1fd9583c4ce07a9d8f82c732e1836
I love block chain9->2502ecec685159e5d18196d1a870b7b0d458d9b161c721299604e7fba7fac486


接下来给定目标$y$，计算满足条件$H(z)<y$的nonce

In [2]:
# 给定目标y为一个大整数
y = 1 << 248
y

452312848583266388373324160190187140051835877600158453279131187530910662656

In [3]:
# 将z的哈希值转换为整数，如果小于目标y就退出循环，否则nonce加1
nonce = 0
while int(sha256d(s + str(nonce)), 16) >= y:
    nonce += 1
# 满足条件的nonce
nonce

609

In [4]:
# 将nonce加1计算一次，不满足条件
int(sha256d(s + str(nonce + 1)), 16) < y

False

“挖矿”所需的“难题”由“矿工”用候选区块构造的。

In [5]:
from simchain import Vin, Vout, Tx, Block
vin = Vin(to_spend=None, signature=b'0', pubkey=None)
vout = Vout(to_addr='1', value=100)
tx = Tx([vin], [vout])
# 创建一个区块
block = Block(version=1.0, prev_block_hash=None, timestamp=850114, bits=20, nonce=12345, txs=[tx])
# 获取区块头
block.header()

'1.0None850114205b7aa75ea52c668b27693c9af16229e65c98af92392d96cab82957b6c75d9bf012345'

In [6]:
# 改变nonce值
block.header(11)

'1.0None850114205b7aa75ea52c668b27693c9af16229e65c98af92392d96cab82957b6c75d9bf011'

难度bits用于计算目标值y，字符串s是区块头中不包括nonce的部分，z就是区块头。

In [7]:
from simchain.ecc import sha256d

# 通过难度位数计算目标值
def caculate_target(bits):
    return (1 << (256 - bits))

# 定义“挖矿”函数，block：候选区块
def mine(block):
    # 初始化随机数nonce
    nonce = 0
    # 根据候选区块中的难度位数计算目标值
    target = caculate_target(block.bits)
    # 计算区块的梅克尔树根哈希值
    merkle_root_hash = block.get_merkle_root()
    # 直到nonce满足条件才退出循环
    while int(sha256d(block.header(nonce, merkle_root_hash)), 16) >= target:
        nonce += 1

    return nonce

In [8]:
from simchain.consensus import mine
# 计算2008865次时，满足条件
mine(block)

2008865

### 难度调整

通过调整目标值y，从而保证让网络中计算最快的节点在10分钟左右才能计算出满足条件的nonce。实际上，难度调整的本质是增大和减小目标y的值。

Simchain中的难度计算函数caculate_target()，难度位数bits增加，则“挖矿”难度增加，bits减小，则“挖矿”难度降低。

In [9]:
from simchain.consensus import caculate_target
y1 = caculate_target(20)
y2 = caculate_target(21)
y1 > y2

True

在Simchain中通过当前区块难度计算下一个区块难度的函数示例如下：

In [10]:
# local_time：区块产生累计时间，prev_height：前区块高度，prev_bits：前区块难度位数
# 区块产生累计时间指每产生一块区块花费时间的累计集合
# 比如产生3个区块花费时间分别为3、4、5分钟，则集合为｛0,3,7,12｝，是Simchain中的本地时间
# 比特币中可以通过区块时间戳来确定
def calculate_next_block_bits(local_time, prev_height, prev_bits):
    
    # Params.TOTAL_BLOCKS：每产生多少区块后，系统自动调整难度1次，简称难度调整间隔
    # 比特币为2016块，Simchain为20块，如果当前区块高度不为难度调整间隔的倍数，难度不调整
    flag = (prev_height + 1) % Params.TOTAL_BLOCKS
    if flag != 0:
        return prev_bits

    # 计算新一轮难度调整间隔内所消耗的实际总时间
    # 比如比特币为高度1/2017~高度2016/4032花费的总时间
    count = ((prev_height + 1) // Params.TOTAL_BLOCKS) * Params.TOTAL_BLOCKS
    actual_time_taken = local_time[:prev_height] - local_time[:count]

    # 如果实际消耗时间比期望时间小，则增加难度
    # 比特币对的期望时间是2016*10分钟
    if actual_time_taken < Params.PERIOD_FOR_TOTAL_BLOCKS:
        return prev_bits + 1
    # 如果实际消耗时间比期望时间大，则降低难度 
    elif actual_time_taken > Params.PERIOD_FOR_TOTAL_BLOCKS:
        return prev_bits - 1
    # 几乎不可能发生
    else:
        return prev_bits